In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# 1. DF_WC [ GWC, NWC, RF, EPT, DISC, availDISC]

# Define files

In [2]:
file_nwc = '1.exportNWC_v2.xlsx'
file_gwc = '1.exportGWC_v2.xlsx'
file_alert = '_Alerts Data .xlsx'
file_strat = 'all_strategy.xlsx'
file_disc = 'discounting.xlsx'
file_availdisc = 'Potential Discounting by contract.xlsx'

file_pl = 'e931a Margin per ton v10.xlsx'
file_strat = 'all_strategy.xlsx'

## Gross Working Capital

In [3]:
df_gwc = pd.read_excel(file_gwc,converters={'Company Code': str,'Strategy ID': str })
cols_gwc = list(df_gwc.columns)
cols_gwc
df_gwc.columns = cols_gwc[0:6] + [x[7:17] for x in cols_gwc[6:]]


## Net Working Capital

In [4]:
df_nwc = pd.read_excel(file_nwc,converters={'Company Code': str,'Strategy ID': str })
cols_nwc = list(df_nwc.columns)
df_nwc.columns = cols_nwc[0:6] + [x[7:17] for x in cols_nwc[6:]]

## Reverse Factoring

In [5]:
df_rf = pd.read_excel(file_alert, sheetname='Rf data',converters={'Company Code': str,'Strategy ID': str })

In [6]:
date_cols = list(df_gwc.columns[6:])

In [7]:
# calculation of reverse factoring per day
for date in date_cols:
    df_rf[date] = np.where((df_rf['Original payment date'] <= datetime.strptime(date,'%d.%m.%Y')) &
                              (df_rf['Amended Due Date'] > datetime.strptime(date,'%d.%m.%Y')),
                             df_rf['Amount'],
                             0)

In [8]:
# ensuring that both data frames have the same columns
cols_gwc = df_gwc.columns
df_rf = df_rf[cols_gwc]

## Extended Payment Terms

In [9]:
df_ext = pd.read_excel(file_alert, sheetname='DataNarlLukoil',converters={'Company Code': str,'Strategy': str })

In [10]:
# changing column labels
list_cols = list(df_ext.columns)
list_cols[3] = 'standard due date'
list_cols[4] = 'number days'
list_cols[13] = 'Strategy ID'
list_cols[20] = 'category'
list_cols[21] = 'weight'
df_ext.columns = list_cols

In [11]:
# filtering columns
df_ext = df_ext.loc[:,'Delivery ID':'Internal type']

In [12]:
# fill the extended payment terms per day
for date in date_cols:
    df_ext[date] = np.where((df_ext['FIN due date'] > datetime.strptime(date,'%d.%m.%Y')) &
                              (df_ext['standard due date'] <= datetime.strptime(date,'%d.%m.%Y')) &
                            (df_ext['In/Out'] == 'Sell') &
                             (df_ext['Internal type'].isnull() == True) &
                            (df_ext['number days'] > 0),
                             df_ext['Final Amount']*-1,
                             0)

In [13]:
# ensuring that both data frames have the same columns
df_ext = df_ext[cols_gwc]

# replacing EUR by USD

df_ext['Currency'] = 'USD'

#df_ext = df_ext[cols_gwc]

# Discounting

In [14]:
df_disc = pd.read_excel(file_disc,converters={'Company code': str,'StrategyID': str })

In [15]:
# changing column labels
list_cols_disc = list(df_disc.columns)
list_cols_disc[2] = 'Company Code'
list_cols_disc[6] = 'Global Book name'
list_cols_disc[7] = 'Portfolio name'
list_cols_disc[9] = 'Strategy ID'
df_disc.columns = list_cols_disc
df_disc['Currency'] = 'USD'
df_disc['Global Book name'] = df_disc['Global Book name'].str.upper()

In [16]:
# fill the discounting per day
for date in date_cols:
    df_disc[date] = np.where((df_disc['Value Date 2018'] <= datetime.strptime(date,'%d.%m.%Y')) &
                              (df_disc['Due Date 2018'] > datetime.strptime(date,'%d.%m.%Y')),
                             df_disc['WCR amount'],
                             0)

In [17]:
# replacing Middle Distillates by DISTILLATES

df_disc['Global Book name'] = np.where(df_disc['Global Book name'] == 'MIDDLE DISTILLATES',
                                      'DISTILLATES', df_disc['Global Book name'])


# replacing Naphta by NAPHTHA

df_disc['Global Book name'] = np.where(df_disc['Global Book name'] == 'NAPHTA',
                                      'NAPHTHA', df_disc['Global Book name'])

In [18]:
# ensuring that both data frames have the same columns
df_disc = df_disc[cols_gwc]

# Available Discounting

In [19]:
df_availdisc =  pd.read_excel(file_availdisc,sheetname='PotentialDisc',
                              converters={'Company Code': str,'Strategy ID': str }, skiprows=5)

In [20]:
# Title transfer date and Fin due date need to be datetime type
df_availdisc.dtypes

Row Labels                      int64
Counterparty2                   int64
Title transfer date    datetime64[ns]
FIN due date           datetime64[ns]
Delivery ID                    object
Sum of Final Amount           float64
Securitisation                  int64
Initial Date           datetime64[ns]
End Date               datetime64[ns]
Days                            int64
Value Date 2018        datetime64[ns]
Due Date 2018          datetime64[ns]
Calc                          float64
Company Code                   object
Global Book name               object
Line of Business               object
Portfolio name                 object
Strategy ID                    object
Currency                       object
dtype: object

In [21]:
# fill the discounting per day
for date in date_cols:
    df_availdisc[date] = np.where((df_availdisc['Value Date 2018'] <= datetime.strptime(date,'%d.%m.%Y')) &
                              (df_availdisc['Due Date 2018'] > datetime.strptime(date,'%d.%m.%Y')) &
                                  (df_availdisc['Sum of Final Amount'] < -10000000) &
                                  (df_availdisc['Securitisation'] == 0) &
                                  ((df_availdisc['FIN due date'] - df_availdisc['Title transfer date']).dt.days >=10),
                             df_availdisc['Sum of Final Amount']*-1,
                             0)

In [22]:
# ensuring that both data frames have the same columns
df_availdisc = df_availdisc[cols_gwc]

In [23]:

df_availdisc = df_availdisc.fillna(value = 0)
df_availdisc = df_availdisc.groupby(['Strategy ID','Company Code','Global Book name',
                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
# ensuring that both data frames have the same columns - order
df_availdisc = df_availdisc[cols_gwc]

## Cleaning NA's and groupby strategy

In [24]:

# removing sensitive case in GB
df_gwc['Global Book name'] = df_gwc['Global Book name'].str.upper()
df_nwc['Global Book name'] = df_nwc['Global Book name'].str.upper()
df_rf['Global Book name'] = df_rf['Global Book name'].str.upper()
df_ext['Global Book name'] = df_ext['Global Book name'].str.upper()
df_disc['Global Book name'] = df_disc['Global Book name'].str.upper()
df_availdisc['Global Book name'] = df_availdisc['Global Book name'].str.upper()


# removing sensitive case in LOB
df_gwc['Line of Business'] = df_gwc['Line of Business'].str.upper()
df_nwc['Line of Business'] = df_nwc['Line of Business'].str.upper()
df_rf['Line of Business'] = df_rf['Line of Business'].str.upper()
df_ext['Line of Business'] = df_ext['Line of Business'].str.upper()
df_disc['Line of Business'] = df_disc['Line of Business'].str.upper()
df_availdisc['Line of Business'] = df_availdisc['Line of Business'].str.upper()


# ensuring that all nan are replaced by zeros and groupby strategy

# gwc
df_gwc = df_gwc.fillna(value = 0)
#df_gwc = df_gwc.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_gwc = df_gwc[cols_gwc]

# nwc
df_nwc = df_nwc.fillna(value = 0)
#df_nwc = df_nwc.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_nwc = df_nwc[cols_gwc]


# rf
df_rf = df_rf.fillna(value = 0)
#df_rf = df_rf.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_rf = df_rf[cols_gwc]

# ext
df_ext = df_ext.fillna(value = 0)
#df_ext = df_ext.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_ext = df_ext[cols_gwc]

# disc
df_disc = df_disc.fillna(value = 0)
#df_disc = df_disc.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_disc = df_disc[cols_gwc]

# availdisc
df_availdisc = df_availdisc.fillna(value = 0)
#df_availdisc = df_availdisc.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
df_availdisc = df_availdisc[cols_gwc]

## Securitization (not active)

In [25]:
#df_nwc_negative = df_nwc.copy()
#s = df_nwc_negative.select_dtypes(include=[np.number])*-1
#df_nwc_negative[s.columns] = s

In [26]:
#df_disc_negative = df_disc.copy()
#s = df_disc_negative.select_dtypes(include=[np.number])*-1
#df_disc_negative[s.columns] = s

In [27]:
# groupby: SECU = GWC - NWC - DISC
#df_secu = pd.concat([df_gwc, df_nwc_negative, df_disc_negative])
#df_secu = df_secu.fillna(value = 0)

#df_secu['Global Book name'] = df_secu['Global Book name'].str.upper()
#df_secu['Line of Business'] = df_secu['Line of Business'].str.upper()

#df_secu = df_secu.groupby(['Strategy ID','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()
#df_secu = df_secu[cols_gwc]

## Joining the data

In [28]:

# add category to each dataframe
df_gwc['Category'] = 'GWC'
df_nwc['Category'] = 'NWC'
df_rf['Category'] = 'RF'
df_ext['Category'] = 'EPT'
df_disc['Category'] = 'DISC'
df_availdisc['Category'] = 'AvailDISC'
#df_secu['Category'] = 'SECU'

# send the last column to the first
df_gwc = df_gwc.set_index('Category').reset_index()
df_nwc = df_nwc.set_index('Category').reset_index()
df_rf = df_rf.set_index('Category').reset_index()
df_ext = df_ext.set_index('Category').reset_index()
df_disc = df_disc.set_index('Category').reset_index()
df_availdisc = df_availdisc.set_index('Category').reset_index()
#df_secu = df_secu.set_index('Category').reset_index()


In [29]:
#concatenating the dataframes
df_wc = pd.concat([df_gwc, df_nwc, df_rf, df_ext, df_disc, df_availdisc])

# 2. Efficiency reports portfolio adjustments

In [30]:
strat = pd.read_excel(file_strat, dtype=str)

# create a column with portfolio add-ons
strat['Portfolio_adj1'] = strat['Portfolio name']


# Strategies containing "NICO" on "T-Feed"portfolio are assigned to Portfolio "T-Feed NICO"

strat['Portfolio_adj1'] = np.where((strat['Strategy name'].str.contains('NICO') == True) & (strat['Portfolio name'] == 'T-Feed'),
                                  'T-Feed NICO', strat['Portfolio_adj1'])


# Books containing "T-Feed Storages" on "T-Feed" Portfolio are assigned to Portfolio "T-Feed Storage"

strat['Portfolio_adj1'] = np.where((strat['Book Name'].str.contains('T-Feed Storages') == True) & (strat['Portfolio name'] == 'T-Feed'),
                                  'T-Feed Storage', strat['Portfolio_adj1'])


# Books containing "T-Feed US ARB" on "T-Feed" Portfolio are assigned to Portfolio "T-Feed US ARB"

strat['Portfolio_adj1'] = np.where((strat['Book Name'].str.contains('T-Feed US ARB') == True) & (strat['Portfolio name'] == 'T-Feed'),
                                  'T-Feed US ARB', strat['Portfolio_adj1'])


# Portfolios "Term Contract" and "Term Contracts" on "DISTILLATES" Global Book are assigned to Portfolio "Storages NWE ULSD"

strat['Portfolio_adj1'] = np.where((strat['Global Book name'] == 'DISTILLATES') & (strat['Portfolio name'].str.contains('Term Contract')),
                                  'Storages NWE ULSD', strat['Portfolio_adj1'])


# Portfolio "T-Feed Paper" on "0100" Company Code are assigned to Portfolio "T-Feed Cargoes"

strat['Portfolio_adj1'] = np.where((strat['Company code'] == '0100') & (strat['Portfolio name'] == 'T-Feed Paper'),
                                  'T-Feed Cargoes', strat['Portfolio_adj1'])


# Portfolio adj "T-Feed" on "0100" Company Code are assigned to Portfolio "T-Feed Cargoes"

strat['Portfolio_adj1'] = np.where((strat['Company code'] == '0100') & (strat['Portfolio_adj1'] == 'T-Feed'),
                                  'T-Feed Cargoes', strat['Portfolio_adj1'])


# Line of business LAP Cracked into Portfolio "T-Cracked East ARB"

strat['Portfolio_adj1'] = np.where(strat['Line of Business'] == 'LAP Cracked',
                                  'T-Cracked East ARB', strat['Portfolio_adj1'])



#send portfolios to margin calls/costs/unassigned based on arguments

strat['Portfolio_adj2'] = strat['Portfolio_adj1'].str.lower()
strat['Portfolio_adj2'] = np.where((strat['Portfolio_adj2'].str.contains('term contract')) 
                         | (strat['Portfolio_adj2'].str.contains('paper'))
                         | (strat['Portfolio_adj2'].str.contains('cost'))          
                         | (strat['Portfolio_adj2'].str.contains('emission'))                   
                         | (strat['Portfolio_adj2'].str.contains('default'))                   
                         | (strat['Portfolio_adj2'].str.contains('derivative'))
                         | (strat['Portfolio_adj2'].str.contains('intel'))
                         | (strat['Portfolio_adj2'].str.contains('hedg'))
                         | (strat['Portfolio_adj2'].str.contains('accounting'))          
                         | (strat['Portfolio_adj2'].str.contains('forward'))                    
                         ,strat['Global Book name'] + ' Margin Call/Paper/Costs', strat['Portfolio_adj1'])


In [31]:
# set index in strat so we can do the vlookup/map
strat = strat.set_index('Strategy')

# assign the adjusted portfolios to working capital data
df_wc['Portfolio_ER'] = df_wc['Strategy ID'].map(strat['Portfolio_adj2'])

# set and reset index so the new column does not stay at the right
df_wc = df_wc.set_index('Portfolio_ER')
df_wc = df_wc.reset_index()

In [32]:
# list portfolios to allocate

print(list(df_wc[df_wc['Strategy ID'] == '0']['Portfolio name'].unique()))


[0, 'GVA Crude Oil Accounting', 'GVA Distillates Accounting', 'GVA Gasoline Accounting', 'GVA Cracked Accounting', 'GVA Naphtha Accounting', 'LAP Crude Accounting', 'LAP Distillates Accounting', 'LAP Gasoline Accounting', 'LAP Cracked Accounting', 'LAP Naphtha Accounting', 'LME Distillates Accounting', 'LPA Crude Accounting', 'NARL REFINING', 'LPA Distillates Accounting', 'LPA USGC Blending Accounting', 'LPA Naphtha Accounting']


#### Adjustments of portfolios

In [33]:
# all in company "0600" goes to "Naphtha"
df_wc['Portfolio_ER'] = np.where(df_wc['Company Code'] == '0600','Naphtha', df_wc['Portfolio_ER'])

# company "0700" in DISTILLATES to LBBV Storages
df_wc['Portfolio_ER'] = np.where((df_wc['Company Code'] == '0700') &
                                  (df_wc['Global Book name'] == 'DISTILLATES'), 'LBBV Storages',
                                  df_wc['Portfolio_ER'])

# company "0700" in HEAVY to LBBV Physical Portfolio
df_wc['Portfolio_ER'] = np.where((df_wc['Company Code'] == '0700') &
                                  (df_wc['Global Book name'] == 'HEAVY'), 'Physical Portfolio',
                                  df_wc['Portfolio_ER'])


# portfolio name "Storages MED" in DISTILLATES
df_wc['Portfolio_ER'] = np.where((df_wc['Strategy ID'] == '0') &
                                 (df_wc['Global Book name'] == 'DISTILLATES') &
                                 (df_wc['Portfolio name'] == 'Storages MED'),'Storages MED',
                                 df_wc['Portfolio_ER'])


# portfolio name "NARL REFINING" to NARL REFINING
df_wc['Portfolio_ER'] = np.where((df_wc['Strategy ID'] == '0') &
                                 (df_wc['Portfolio name'] == 'NARL REFINING'),'NARL REFINING',
                                 df_wc['Portfolio_ER'])


# all the remaining are assigned to "Global Book name" + " Margin Call/Paper/Costs
df_wc['Portfolio_ER'] = np.where(df_wc['Portfolio_ER'].isnull() == True,
                                  df_wc['Global Book name'].astype(str) + ' Margin Call/Paper/Costs',
                                 df_wc['Portfolio_ER'])

# Euroeco fuels
df_wc['Portfolio_ER'] = np.where((df_wc['Strategy ID'] == '505369') |
                                 (df_wc['Strategy ID'] == '509185') &
                                 (df_wc['Global Book name'] == 'HEAVY'),'EUROECO FUELS',
                                 df_wc['Portfolio_ER'])


### Separate the JV's from Global Books

In [34]:
# JV's

df_wc['Global Book name'] = np.where(df_wc['Line of Business'].str.contains('IPT') == True,
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Global Book name'] = np.where(df_wc['Line of Business'].str.contains('NARL') == True,
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Global Book name'] = np.where(df_wc['Line of Business'].str.contains('GNPC') == True,
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Global Book name'] = np.where(df_wc['Line of Business'].str.contains('MRS JV') == True,
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Global Book name'] = np.where(df_wc['Portfolio name'].str.contains('FalconRain') == True,
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Global Book name'] = np.where(df_wc['Portfolio_ER'] == 'EUROECO FUELS',
                                    'JVs',
                                    df_wc['Global Book name'])


df_wc['Global Book name'] = np.where(df_wc['Line of Business'] == 'LPA BASF HEDGE',
                                    'JVs',
                                    df_wc['Global Book name'])

df_wc['Portfolio_ER'] = np.where(df_wc['Line of Business'] == 'LPA BASF HEDGE',
                                    df_wc['Portfolio name'],
                                    df_wc['Portfolio_ER'])

In [35]:
# list portfolios to allocate

print(list(df_wc[df_wc['Strategy ID'] == '0']['Portfolio_ER'].unique()))


['0 Margin Call/Paper/Costs', 'ASSET TRADING Margin Call/Paper/Costs', 'CRUDE Margin Call/Paper/Costs', 'DERIVATIVES Margin Call/Paper/Costs', 'DISTILLATES Margin Call/Paper/Costs', 'GASOLINE Margin Call/Paper/Costs', 'HEAVY Margin Call/Paper/Costs', 'NAPHTHA Margin Call/Paper/Costs', 'OTHER Margin Call/Paper/Costs', 'PETCHEM Margin Call/Paper/Costs', 'REFINING Margin Call/Paper/Costs', 'LCA Margin Call/Paper/Costs', 'Naphtha', 'LBBV Storages', 'Physical Portfolio', 'ACCOUNTING Margin Call/Paper/Costs', 'NARL REFINING']


In [36]:
# add book-key column

df_wc['Book - Key'] = df_wc['Strategy ID'].map(strat['Book - Key'])
df_wc.set_index('Book - Key', inplace=True)
df_wc.reset_index(inplace=True)

## EXPORT DATA

In [37]:
# export data


# Month

#df_wc['JAN_avg'] = df_wc.loc[:,'01.01.2018':'31.01.2018'].mean(axis=1)
#df_wc['FEB_avg'] = df_wc.loc[:,'01.02.2018':'28.02.2018'].mean(axis=1)
#df_wc['MAR_avg'] = df_wc.loc[:,'01.03.2018':'31.03.2018'].mean(axis=1)

# YTD by month
#df_wc['JAN_avg_YTD'] = df_wc.loc[:,'01.01.2018':'31.01.2018'].mean(axis=1)
#df_wc['FEB_avg_YTD'] = df_wc.loc[:,'01.01.2018':'28.02.2018'].mean(axis=1)
#df_wc['MAR_avg_YTD'] = df_wc.loc[:,'01.01.2018':'31.03.2018'].mean(axis=1)

# Quarter
#df_wc['Q1_avg'] = df_wc.loc[:,'01.01.2018':'31.03.2018'].mean(axis=1)

# YTD
#df_wc['YTD_avg'] = df_wc.loc[:,'01.01.2018':'24.04.2018'].mean(axis=1)



#writer = pd.ExcelWriter('database_wc.xlsx')
#df_wc.to_excel(writer, 'data_wc')

## Efficiency Report

In [38]:
df_wc_er = df_wc.copy()

# divide the numerical columns (date columns) by 1000000
df_wc_er[date_cols] = df_wc_er[date_cols]/1000000

In [39]:
# groupby portfolio

#df_wc_er = df_wc_er.groupby(['Book - Key','Portfolio_ER','Category','Company Code','Global Book name',
#                                     'Line of Business','Portfolio name','Currency'])[date_cols].sum().reset_index()

In [40]:

# Month

df_wc_er['JAN_avg'] = df_wc_er.loc[:,'01.01.2018':'31.01.2018'].mean(axis=1)
df_wc_er['FEB_avg'] = df_wc_er.loc[:,'01.02.2018':'28.02.2018'].mean(axis=1)
df_wc_er['MAR_avg'] = df_wc_er.loc[:,'01.03.2018':'31.03.2018'].mean(axis=1)
df_wc_er['APR_avg'] = df_wc_er.loc[:,'01.04.2018':'30.04.2018'].mean(axis=1)
df_wc_er['MAY_avg'] = df_wc_er.loc[:,'01.05.2018':'31.05.2018'].mean(axis=1)
df_wc_er['JUN_avg'] = df_wc_er.loc[:,'01.06.2018':'30.06.2018'].mean(axis=1)

# YTD by month
df_wc_er['JAN_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'31.01.2018'].mean(axis=1)
df_wc_er['FEB_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'28.02.2018'].mean(axis=1)
df_wc_er['MAR_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'31.03.2018'].mean(axis=1)
df_wc_er['APR_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'30.04.2018'].mean(axis=1)
df_wc_er['MAY_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'31.05.2018'].mean(axis=1)
df_wc_er['JUN_avg_YTD'] = df_wc_er.loc[:,'01.01.2018':'30.06.2018'].mean(axis=1)

# Quarter
df_wc_er['Q1_avg'] = df_wc_er.loc[:,'01.01.2018':'31.03.2018'].mean(axis=1)
df_wc_er['Q2_avg'] = df_wc_er.loc[:,'01.04.2018':'30.06.2018'].mean(axis=1)


# YTD
df_wc_er['YTD_avg'] = df_wc_er.loc[:,'01.01.2018':'30.06.2018'].mean(axis=1)

In [41]:
# filter data
#df_wc_er = df_wc_er.loc[(df_wc_er['Global Book name'] == 'CRUDE') |
#                       (df_wc_er['Global Book name'] == 'HEAVY') |
#                       (df_wc_er['Global Book name'] == 'DISTILLATES') |
#                       (df_wc_er['Global Book name'] == 'GASOLINE') |
#                       (df_wc_er['Global Book name'] == 'NAPHTHA') |
#                       (df_wc_er['Global Book name'] == 'JVs')]

### Export data

In [42]:
# export data

#writer = pd.ExcelWriter('database_portfolio.xlsx')
#df_wc_er.to_excel(writer, 'data')

## Import PL


In [43]:
file_pl = 'e931a Margin per ton v10.xlsx'
#file_strat = 'all_strategy.xlsx'
strat.reset_index(inplace=True)

# import pl
df_pl = pd.read_excel(file_pl, sheetname='e931a PL by books(historical)', skiprows=5)

# import strat
#strat = pd.read_excel(file_strat, dtype=str)

# replacing nan by zero

df_pl.fillna(0,inplace=True)


# deleting all the rows containing total and drop them

condition1 = df_pl[df_pl['Global Book'].str.contains('Total') == True]
df_pl.drop(condition1.index, inplace=True)


# deleting all rows with Global Book and drop them

condition2 = df_pl[df_pl['Global Book'] == 'Global Book']

df_pl.drop(condition2.index, inplace = True)


# filter columns

df_pl = df_pl[['MST','Book - Key', 'Adj Amount, USD']]


# change columns name

df_pl.columns = ['MST','Book - Key', 'YTD']
df_pl['YTD'] = df_pl['YTD']/1000000

# mapping

df_pl['Company Code'] = df_pl['Book - Key'].map(
    strat.drop_duplicates(subset='Book - Key').set_index('Book - Key')['Company code'])


df_pl['Global Book name'] = df_pl['Book - Key'].map(
    strat.drop_duplicates(subset='Book - Key').set_index('Book - Key')['Global Book name'])

df_pl['Global Book name'] = df_pl['Global Book name'].str.upper()

df_pl['Line of Business'] = df_pl['Book - Key'].map(
    strat.drop_duplicates(subset='Book - Key').set_index('Book - Key')['Line of Business'])

df_pl['Line of Business'] = df_pl['Line of Business'].str.upper()

df_pl['Portfolio name'] = df_pl['Book - Key'].map(
    strat.drop_duplicates(subset='Book - Key').set_index('Book - Key')['Portfolio name'])

df_pl['Currency'] = 'USD'

#df_pl['Category'] = 'PL'

df_pl['Portfolio_ER'] = df_pl['Book - Key'].map(strat.drop_duplicates(subset='Book - Key').set_index('Book - Key')['Portfolio_adj2'])



col_order = ['Book - Key','Portfolio_ER','MST','Company Code','Global Book name',
             'Line of Business','Portfolio name', 'Currency','YTD']


df_pl = df_pl[col_order]


# filter data
#df_pl = df_pl.loc[(df_pl['Global Book name'] == 'CRUDE') |
#                       (df_pl['Global Book name'] == 'HEAVY') |
#                       (df_pl['Global Book name'] == 'DISTILLATES') |
#                       (df_pl['Global Book name'] == 'GASOLINE') |
#                       (df_pl['Global Book name'] == 'NAPHTHA')]

# export Working Capital and PL

In [45]:
# WC is average, PL is sum

# export data

writer = pd.ExcelWriter('efficiency_data_all v2.xlsx')
df_wc_er.to_excel(writer, 'data_wc', index=False)
df_pl.to_excel(writer, 'data_pl', index=False)